In [1]:
import sys
sys.path.append('..')
from tool import *
from preprocess.DDRF import resample_wb_fraction 
def splice(files,output_file,pixel_size=0.0003,band_num=1,type='int32',masked=None):
    # print(files[0])
    MinX, MaxY, MaxX, MinY = get_extent(files[0])

    for fn in files[1:]:
        minX, maxY, maxX, minY = get_extent(fn)
        MinX = min(MinX, minX)
        MaxY = max(MaxY, maxY)
        MaxX = max(MaxX, maxX)
        MinY = min(MinY, minY)


    ds = gdal.Open(files[0])
    transform = ds.GetGeoTransform()
    pixelWidth = transform[1]
    pixelHeight = transform[5]
    #获取输出图像的行与列
    cols = int((MaxX - MinX) / pixel_size)
    rows = int((MaxY - MinY) / pixel_size)
    # print(cols)
    NP2GDAL_CONVERSION = {
      "uint8": 1,
      "int8": 1,
      "uint16": 2,
      "int16": 3,
      "uint32": 4,
      "int32": 5,
      "float32": 6,
      "float64": 7,
      "complex64": 10,
      "complex128": 11,
    }
    driver = gdal.GetDriverByName('GTiff')
    dsOut = driver.Create(output_file,
                            cols,rows,1,5,
                            ['COMPRESS=LZW','BIGTIFF=YES'])
    bandOut = dsOut.GetRasterBand(1)

    # 设置输出图像的几何信息和投影信息
    geotransform = [MinX, pixel_size, 0, MaxY, 0, pixel_size*(-1)]
    dsOut.SetGeoTransform(geotransform)
    dsOut.SetProjection(ds.GetProjection())
    # label=np.ones([rows,cols],dtype=type)*(-1001)
    label=np.zeros([rows,cols],dtype=type)
    n=0
    for fn in tqdm(files, desc=f'Band {band_num} '):
        ds = gdal.Open(fn)
        transform = ds.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]

        rows = ds.RasterYSize
        cols = ds.RasterXSize
        transform = ds.GetGeoTransform()
        minX = transform[0]
        maxY = transform[3]
        pixelWidth = transform[1]
        pixelHeight = transform[5]#是负值（important）

        maxX = minX + (cols * pixelWidth)
        minY = maxY + (rows * pixelHeight)

        xOffset = round((minX - MinX) / pixel_size)
        yOffset = round((maxY - MaxY) / (pixel_size*(-1)))
        cols=int(cols*pixelWidth/pixel_size)
        rows=int(rows*pixelHeight/(pixel_size*(-1)))
        band = ds.GetRasterBand(band_num)
        data = band.ReadAsArray()
        if masked is not None:
            data[data>=masked]=np.nan
        label_part=label[yOffset:yOffset+rows,xOffset:xOffset+cols]
        label[yOffset:yOffset+rows,xOffset:xOffset+cols]=data
        n+=1
    label[np.isnan(label)]=0
    label[label>=1000]=1000
    label[label<=-1000]=-1000
    label*=100
    label=label.astype('int32')
    dsOut.GetRasterBand(1).WriteArray(label)
    dsOut.FlushCache()
    del dsOut


In [ ]:
for t in ['W','B']:
    for b in ['NDVI','SSI','NDMI']:
        for m in range(1,13):
            try:
                print(t,b,m)
                fs=glob(f'/ssd_hzy/G2/{t}_{b}{m}-*')

                splice(fs,f'/ssd_hzy/G3/{t}_{b}{m}.tif',pixel_size=0.004491576420597608,band_num=1,type='float32')
            except:
                print(t,b,m)
                fs=glob(f'/ssd_hzy/G2/{t}_{b}{m}-*')

                splice(fs,f'/ssd_hzy/G3/{t}_{b}{m}.tif',pixel_size=0.004491576420597608,band_num=1,type='float32')


In [ ]:
def bm(b,m):
    print(b,m)
    W=rasterio.open(f'/data2/hzy/ssd_hzy/G3/W_{b}{m}.tif').read(1)
    B=rasterio.open(f'/data2/hzy/ssd_hzy/G3/B_{b}{m}.tif').read(1)
    WB_sky_fraction=torch.load("/data/hk/albedo/white_sky_fraction/white_sky_fraction2.pth")
    wb=(WB_sky_fraction[(2020-2001)*12+m-1]+WB_sky_fraction[(2001-2001)*12+m-1])/2
    wb=(WB_sky_fraction[(2020-2001)*12+m-1]+WB_sky_fraction[(2001-2001)*12+m-1])/2
    wb_resample=resample_wb_fraction(wb,width=80152,height=40076,resample_alg = gdalconst.GRIORA_NearestNeighbour)
    wb_resample[wb_resample==2]=np.nan


    W[np.isnan(W)]=0
    B[np.isnan(B)]=0
    a=(W*wb_resample+B*(1-wb_resample))/100000

    tif_save(a,f'/data2/hzy/ssd_hzy/G3/{b}{m}_2.tif',global_trf,p='4326')
    del a,B,W,wb_resample
    gc.collect()
    img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}{m}_2.tif').read(1)
    plt.imshow(img,vmin=-0.05,vmax=0.05,cmap='seismic')
    plt.colorbar()
    plt.show()
from tool import *
for b in ['NDVI','SSI','NDMI']:
    for m in range(1,13):
        bm(b,m)

In [2]:
from tool import *
sza,sza_l=torch.load('/data/hk/albedo/SZA.pth')
lc01=rasterio.open("/data/hk/albedo/landcover/2001_landcover/2001_landcover.tif").read(1)
lc20=rasterio.open("/data/hk/albedo/landcover/2020_landcover/2020_landcover.tif").read(1)
z=np.zeros(lc01.shape)
z[lc01==lc20]=1
z[lc01==20]=0
z[lc01==15]=0
z[lc01==16]=0
z[lc01==17]=0
del lc01,lc20
gc.collect()
for b in ['NDVI','SSI','NDMI']:
    count=np.zeros((40076, 80152))
    all=np.zeros((40076, 80152))
    for m in tqdm(range(1,13)):
        now=datetime.datetime.strptime(f'2001-{m}-1', '%Y-%m-%d')
        month_time=(now+relativedelta(months=1)-now).total_seconds()
        img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}{m}_2.tif').read(1)
        SZA_L=np.ones((40076, 80152))
        SZA_L=SZA_L*(sza_l[m-1].reshape(-1,1))


        img[SZA_L>0]=np.nan
        
        del SZA_L
        gc.collect()
        c='2001'+str(m).zfill(2)
        sc_01=rasterio.open(root_path+f'snow/snow_monthly/{c}.tif').read(1)
        c='2020'+str(m).zfill(2)
        sc_20=rasterio.open(root_path+f'snow/snow_monthly/{c}.tif').read(1)
        sc=(100-np.maximum(sc_01,sc_20))/100

        img*=sc
        del sc_01,sc_20,sc
        gc.collect()
        # raise()
        img*=z
        img[img==0]=np.nan
        
        # tif_save_snowfre(img,f'/data2/hzy/ssd_hzy/G3/{b}{m}_albedo2_multisnow.tif',global_trf,p='4326')
        count[~np.isnan(img)]+=month_time
        img[np.isnan(img)]=0
        img=img*month_time
        all+=img
        # plt.imshow(sc)
        # plt.colorbar()
        # plt.show()
        # raise()
        del img 
        gc.collect()
    all/=count
    # all/=100
    tif_save(all,f'/data2/hzy/ssd_hzy/G3/{b}_v2_kernel_2.tif',global_trf,p='4326')
    # plt.imshow(count)
    # plt.colorbar()
    # plt.show()
    del count,all
    gc.collect()

100%|██████████| 12/12 [13:25<00:00, 67.09s/it]


In [2]:
sza,sza_l=torch.load('/data/hk/albedo/SZA.pth')
lc01=rasterio.open("/data/hk/albedo/landcover/2001_landcover/2001_landcover.tif").read(1)
lc20=rasterio.open("/data/hk/albedo/landcover/2020_landcover/2020_landcover.tif").read(1)
z=np.zeros(lc01.shape)
z[lc01==lc20]=1
z[lc01==20]=0
z[lc01==15]=0
z[lc01==16]=0
z[lc01==17]=0
del lc01,lc20
gc.collect()
for b in ['NDVI','SSI','NDMI']:
    count=np.zeros((40076, 80152))
    all=np.zeros((40076, 80152))
    for m in tqdm(range(1,13)):
        img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}{m}_2.tif').read(1)
        SZA_L=np.ones((40076, 80152))
        SZA_L=SZA_L*(sza_l[m-1].reshape(-1,1))


        img[SZA_L>1]=np.nan
        
        del SZA_L
        gc.collect()
        c='2001'+str(m).zfill(2)
        sc_01=rasterio.open(root_path+f'snow/snow_monthly/{c}.tif').read(1)
        c='2020'+str(m).zfill(2)
        sc_20=rasterio.open(root_path+f'snow/snow_monthly/{c}.tif').read(1)
        sc=(100-np.maximum(sc_01,sc_20))/100

        img*=sc
        del sc_01,sc_20,sc
        gc.collect()
        # raise()
        img*=z
        img[img==0]=np.nan
        
        tif_save(img,f'/data2/hzy/ssd_hzy/G3/{b}{m}_albedo2_multisnow_85_2.tif',global_trf,p='4326')
        count[~np.isnan(img)]+=1
        img[np.isnan(img)]=0
        all+=img
        # plt.imshow(sc)
        # plt.colorbar()
        # plt.show()
        # raise()
        del img 
        gc.collect()
    all/=count
    # all/=100
    tif_save(all,f'/data2/hzy/ssd_hzy/G3/{b}_v2_85_2.tif',global_trf,p='4326')
    # plt.imshow(count)
    # plt.colorbar()
    # plt.show()

100%|██████████| 12/12 [21:19<00:00, 106.63s/it]


In [3]:
sza,sza_l=torch.load('/data/hk/albedo/SZA.pth')
for b in ['SSI','NDMI','NDVI']:
    img_y=np.zeros((40076, 80152)).astype(np.float32)
    all=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}_v2_85_2.tif').read(1)
    for m in tqdm(range(1,13)):
        SZA_L=np.ones((40076, 80152))
        SZA_L=SZA_L*(sza_l[m-1].reshape(-1,1))
        now=datetime.datetime.strptime(f'2001-{m}-1', '%Y-%m-%d')
        month_time=(now+relativedelta(months=1)-now).total_seconds()
        img=rasterio.open(f'/data2/hzy/ssd_hzy/G3/{b}{m}_albedo2_multisnow_85_2.tif').read(1)
        img[img==0]=all[img==0]
        # raise()
        sw=torch.load(f"/data/hk/albedo/kernel/{m}_SW_1degree_kernel2.pth")
        sw=resample_wb_fraction(sw,width=80152,height=40076,resample_alg = gdalconst.GRIORA_NearestNeighbour)
        img[img==0]=np.nan
        img[SZA_L>1]=np.nan
        eg=(-img)*sw*month_time
        eg[np.isnan(eg)]=0
        img_y+=eg
        del img,SZA_L,sw
        gc.collect()
        # raise()
    y_len=365*24*3600
    img_y/=y_len
    tif_save(img_y,f'/data2/hzy/ssd_hzy/G3/{b}_eg_v2_kernel_2.tif',global_trf,p='4326')
    del img_y,all
    gc.collect()
    # plt.imshow(img_y,vmax=10,vmin=-10,cmap='seismic')
    # plt.colorbar()
    # plt.show()

100%|██████████| 12/12 [08:55<00:00, 44.60s/it]
